# First we import the required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!pip install selenium

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


# The data to be used is a csv file uploaded unto my data in the cognitive lab.
We begin by reading the csv data from the source and loading it into a pandas dataframe.

In [17]:
csv_data = r'C:\Users\HARSHAVARDHAN\Desktop\uscities1.csv'
csv_data
df = pd.read_csv(csv_data)
df.head(10)

,City Name,city_ascii,State Id,State Name,County Code,County Name,County Code_all,County Name.1,Latitude,Longitude,population,density,source,military,incorporated,timezone,ranking,zips,id
0,South Creek,South Creek,WA,Washington,53053,Pierce,53053,Pierce,46.9994,-122.3921,2500,125.0,polygon,False,True,America/Los_Angeles,3,98580 98387 98338,1840042075
1,Roslyn,Roslyn,WA,Washington,53037,Kittitas,53037,Kittitas,47.2507,-121.0989,947,84.0,polygon,False,True,America/Los_Angeles,3,98941 98068 98925,1840019842
2,Sprague,Sprague,WA,Washington,53043,Lincoln,53043,Lincoln,47.3048,-117.9713,441,163.0,polygon,False,True,America/Los_Angeles,3,99032,1840021107
3,Gig Harbor,Gig Harbor,WA,Washington,53053,Pierce,53053,Pierce,47.3352,-122.5968,9507,622.0,polygon,False,True,America/Los_Angeles,3,98332 98335,1840019855
4,Lake Cassidy,Lake Cassidy,WA,Washington,53061,Snohomish,53061,Snohomish,48.0639,-122.0920,3591,131.0,polygon,False,True,America/Los_Angeles,3,98223 98258 98270,1840041959
5,Tenino,Tenino,WA,Washington,53067,Thurston,53067,Thurston,46.8537,-122.8607,1830,491.0,polygon,False,True,America/Los_Angeles,3,98589,1840021144
6,Jamestown,Jamestown,WA,Washington,53009,Clallam,53009,Clallam,48.1229,-123.0911,289,191.0,polygon,False,False,America/Los_Angeles,3,98382,1840037668
7,Three Lakes,Three Lakes,WA,Washington,53061,Snohomish,53061,Snohomish,47.9420,-121.9924,3390,112.0,polygon,False,False,America/Los_Angeles,3,98290,1840018375
8,Curlew Lake,Curlew Lake,WA,Washington,53019,Ferry,53019,Ferry,48.7311,-118.6663,573,50.0,polygon,False,True,America/Los_Angeles,3,99166,1840043278
9,Chain Lake,Chain Lake,WA,Washington,53061,Snohomish,53061,Snohomish,47.9038,-121.9861,4280,156.0,polygon,False,True,America/Los_Angeles,3,98290,1840041915


In [18]:
df.shape
print('The dataframe has', df.shape, 'rows and columns respectively.')

The dataframe has (28889, 19) rows and columns respectively.


# Segmenting North Carolina Information
Since the dataframe contains information of the whole of the United States, North Carolina (NC), the State of interest was segmented from the whole and some of the column names renamed.

In [19]:
df.rename(columns={'State Id': 'State', 'County Name':'County', 'County Code':'CountyCode', 'City Name':'Neighborhood'}, inplace=True)
df.head()

,Neighborhood,city_ascii,State,State Name,CountyCode,County,County Code_all,County Name.1,Latitude,Longitude,population,density,source,military,incorporated,timezone,ranking,zips,id
0,South Creek,South Creek,WA,Washington,53053,Pierce,53053,Pierce,46.9994,-122.3921,2500,125.0,polygon,False,True,America/Los_Angeles,3,98580 98387 98338,1840042075
1,Roslyn,Roslyn,WA,Washington,53037,Kittitas,53037,Kittitas,47.2507,-121.0989,947,84.0,polygon,False,True,America/Los_Angeles,3,98941 98068 98925,1840019842
2,Sprague,Sprague,WA,Washington,53043,Lincoln,53043,Lincoln,47.3048,-117.9713,441,163.0,polygon,False,True,America/Los_Angeles,3,99032,1840021107
3,Gig Harbor,Gig Harbor,WA,Washington,53053,Pierce,53053,Pierce,47.3352,-122.5968,9507,622.0,polygon,False,True,America/Los_Angeles,3,98332 98335,1840019855
4,Lake Cassidy,Lake Cassidy,WA,Washington,53061,Snohomish,53061,Snohomish,48.0639,-122.0920,3591,131.0,polygon,False,True,America/Los_Angeles,3,98223 98258 98270,1840041959


In [22]:
NC_data = df[df.State == 'NC']
NC_data.head()

,Neighborhood,city_ascii,State,State Name,CountyCode,County,County Code_all,County Name.1,Latitude,Longitude,population,density,source,military,incorporated,timezone,ranking,zips,id
6677,Trenton,Trenton,NC,North Carolina,37103,Jones,37103,Jones,35.0640,-77.3553,285,484.0,polygon,False,True,America/New_York,3,28585,1840016445
6678,Stanfield,Stanfield,NC,North Carolina,37167,Stanly,37167,Stanly,35.2336,-80.4303,1517,131.0,polygon,False,True,America/New_York,3,28163,1840016365
6679,Sanford,Sanford,NC,North Carolina,37105,Lee,37105,Lee,35.4874,-79.1772,34557,389.0,polygon,False,True,America/New_York,3,27330 27332 27331,1840015421
6680,Cofield,Cofield,NC,North Carolina,37091,Hertford,37091,Hertford,36.3567,-76.9105,382,46.0,polygon,False,True,America/New_York,3,27922,1840017704
6681,Thomasville,Thomasville,NC,North Carolina,37057,Davidson,37057,Davidson,35.8813,-80.0807,26615,612.0,polygon,False,True,America/New_York,3,27360 27361,1840015361


Now the NC data is sliced and the based on the needed columns for the project

In [23]:
cols = [0,2,3,5,6,8]
NC_data = NC_data[NC_data.columns[cols]]
NC_data.head().reset_index()

,index,Neighborhood,State,State Name,County,County Code_all,Latitude
0,6677,Trenton,NC,North Carolina,Jones,37103,35.0640
1,6678,Stanfield,NC,North Carolina,Stanly,37167,35.2336
2,6679,Sanford,NC,North Carolina,Lee,37105,35.4874
3,6680,Cofield,NC,North Carolina,Hertford,37091,36.3567
4,6681,Thomasville,NC,North Carolina,Davidson,37057,35.8813


In [24]:
NC_data.shape
print('The dataframe has', NC_data.shape, 'rows and columns respectively.')

The dataframe has (736, 6) rows and columns respectively.


# Install and inport folium

In [25]:
!pip install folium
import folium # map rendering library
print('Libraries imported.')

Libraries imported.


In [ ]:
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of New York using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, city, county in zip(NC_data['Latitude'], NC_data['Longitude'], NC_data['Neighborhood'], NC_data['County']):
    label = '{}, {}'.format(county, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='brown',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map) 
map

# Segmenting Wake County <br>
Let's simplify the above map and segment and clusters only the neighborhoods in Wake County. So let's slice the original dataframe and create a new dataframe of the Wake County data.

In [27]:
wake_data = NC_data[NC_data.County == 'Wake']
wake_data.drop(wake_data.index[2], inplace=True)
print('The dataframe has', wake_data.shape, 'rows and columns respectively.')
wake_data.reset_index()

The dataframe has (11, 6) rows and columns respectively.


C:\Users\HARSHAVARDHAN\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,index,Neighborhood,State,State Name,County,County Code_all,Latitude
0,6710,Morrisville,NC,North Carolina,Wake,37183,35.8366
1,6722,Knightdale,NC,North Carolina,Wake,37183,35.7917
2,6904,Apex,NC,North Carolina,Wake,37183,35.7246
3,6917,Cary,NC,North Carolina,Wake,37183|37037,35.7817
4,6919,Raleigh,NC,North Carolina,Wake,37183|37063,35.8324
5,6929,Garner,NC,North Carolina,Wake,37183,35.6938
6,7063,Fuquay-Varina,NC,North Carolina,Wake,37183,35.5961
7,7084,Wake Forest,NC,North Carolina,Wake,37183|37069,35.9634
8,7108,Zebulon,NC,North Carolina,Wake,37183,35.8318
9,7289,Wendell,NC,North Carolina,Wake,37183,35.7823


In [ ]:
address = 'Wake, NC'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Wake, NC are {}, {}.'.format(latitude, longitude))

Now let's get the latitude and longitude values of Wake County, NC

In [ ]:
map_wake = folium.Map(location=[Latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(wake_data['Latitude'], wake_data['Longitude'], wake_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=15,
        popup=label,
        color='brown',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_wake)  
    
map_wake


# Visualizing Wake County Sliced Data

In [ ]:

map_wake = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(wake_data['Latitude'], wake_data['Longitude'], wake_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=15,
        popup=label,
        color='brown',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_wake)  
    
map_wake


Wake County Neighborhood Cluster
Now, the next thing to do is to utilize the Foursquare API to explore the wake county neighborhoods and segment them.

Let's define Foursquare Credentials and Version

In [33]:
CLIENT_ID = 'your Foursquare ID'
CLIENT_SECRET = 'your Foursquare Secret'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret



Let's explore the first neighborhood in the Wake County dataframe.

In [ ]:
wake_data.loc[7225,'Neighborhood']
print('The first neighborhood is', wake_data.loc[7225,'Neighborhood'])

Now we get the first neighborhood's latitude and longitude values.¶

In [ ]:
neighborhood_latitude = wake_data.loc[7225, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = wake_data.loc[7225, 'Longitude'] # neighborhood longitude value

neighborhood_name = wake_data.loc[7225, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

# Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [ ]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()

use the get_category_type function to get the category types.

In [ ]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Now data is cleaned and structured into a pandas dataframe.

In [ ]:
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

# Let us explore neighbourhoods

Let's create a function to collect the venues to all the neighborhoods in Wake County

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


let's write the code to run the above function on each neighborhood and create a new dataframe called Wake County_venues.

In [ ]:
wake_venues = getNearbyVenues(names=wake_data['Neighborhood'],
                                   latitudes=wake_data['Latitude'],
                                   longitudes=wake_data['Longitude']
                                  )
print('There are', wake_venues.shape, 'rows and columns of venues and neighborhoods in Wake County.')
wake_venues.head(10)

There are (106, 7) rows and columns of venues and neighborhoods in Wake County.

In [ ]:
plot1=wake_venues["Neighborhood"].value_counts().plot(kind = "bar")
plot1.set_xlabel('Neighborhood')
plot1.set_ylabel('Venue Count')

Let's check how many venues were returned for each neighborhood

In [ ]:
venue_counts = wake_venues.groupby('Neighborhood').count()
venue_counts

Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(wake_venues['Venue Category'].unique())))

There are 62 uniques categories.

Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in wake_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = wake_grouped[wake_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

<b>Let's put that into a pandas dataframe</b> <br>
First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = wake_grouped['Neighborhood']

for ind in np.arange(wake_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(wake_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

In [ ]:
neighborhoods_venues_sorted.shape
print('The dataframe has', neighborhoods_venues_sorted.shape, 'rows and columns respectively.')


The dataframe has (12, 11) rows and columns respectively.

Let's Cluster the Neighborhoods in Wake County.
Run k-means to cluster the neighborhood into 4 clusters.

In [ ]:
# set number of clusters
kclusters = 4

wake_grouped_clustering = wake_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(wake_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
wake_merged = wake_data

# add clustering labels
wake_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
wake_merged = wake_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

wake_merged

# let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(wake_merged['Latitude'], wake_merged['Longitude'], wake_merged['Neighborhood'], wake_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
from folium.plugins import FloatImage
image_file = 'http://printad.us/wp-content/uploads/2018/07/wake-county-zip-code-map-maps-wake-county-north-carolina-map-a-history-of-zip-code-wake-for-557-x-575-pixels.jpg'

FloatImage(image_file, bottom=0, left=70).add_to(map_clusters)
map_clusters

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
#wake_merged.hist(column='Density')
wake_merged.plot(kind='bar',x='Neighborhood',y='Density', color=rainbow[cluster])
plt.show()

# Segmenting Mecklenburg County, NC.


Let's simplify the above map and segment and cluster only the neighborhoods in Mecklenburg County. So let's slice the original dataframe and create a new dataframe of the Mecklenburg County data.

In [ ]:
mecklenburg_data = NC_data[NC_data.County == 'Mecklenburg']
#mecklenburg_data.drop(mecklenburg_data.index[2], inplace=True)
print('The dataframe has', mecklenburg_data.shape, 'rows and columns respectively.')
mecklenburg_data.reset_index()

The dataframe has (10, 6) rows and columns respectively.

In [ ]:
address = 'Mecklenburg, NC'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mecklenburg, NC are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mecklenburg, NC are 35.2356385, -80.8139485.


Let's get the first neighborhood in Mecklenburg County

In [ ]:
print('The first neighborhood in Mecklenburg County is', mecklenburg_data.loc[7182, 'Neighborhood'])

The first neighborhood in Mecklenburg County is Paw Creek

# Let's get the first 100 venues

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

Now let's get the categories using the get_category_type

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Let's clean the data and load into a pandas dataframe

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.

# Let's explore nearby venues

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's write the code to run the above function on each neighborhood and create a new dataframe called mecklenburg_venues.

In [ ]:
mecklenburg_venues = getNearbyVenues(names=mecklenburg_data['Neighborhood'],
                                   latitudes=mecklenburg_data['Latitude'],
                                   longitudes=mecklenburg_data['Longitude']
                                  )

In [ ]:
print('There are', mecklenburg_venues.shape, 'rows and columns venues and neighborhoods in Mecklenburg County.')
mecklenburg_venues.head(10)

There are (71, 7) rows and columns venues and neighborhoods in Mecklenburg County.


Let's look at the number of venues returned for each neighborhood

In [ ]:
mecklenburg_venues.groupby('Neighborhood').count()

# Mecklenburg County Neighborhood Analysis

In [ ]:

# one hot encoding
mecklenburg_onehot = pd.get_dummies(mecklenburg_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mecklenburg_onehot['Neighborhood'] = mecklenburg_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mecklenburg_onehot.columns[-1]] + list(mecklenburg_onehot.columns[:-1])
mecklenburg_onehot = mecklenburg_onehot[fixed_columns]

mecklenburg_onehot.head()

Next, let's group rows by neighborhood by taking the mean of the frequency of occurrence of each category

In [ ]:
mecklenburg_grouped = mecklenburg_onehot.groupby('Neighborhood').mean().reset_index()
mecklenburg_grouped


Let's again print each neighborhood along with the top 5 venues

In [ ]:

for hood in mecklenburg_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = mecklenburg_grouped[mecklenburg_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Let's again put this into a pandas dataframe
Let's write a function to sort venues in decending order

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = mecklenburg_grouped['Neighborhood']

for ind in np.arange(mecklenburg_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mecklenburg_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


# Mecklenburg County Neighborhood Clustering

In [ ]:
# set number of clusters
kclusters = 4

mecklenburg_grouped_clustering = mecklenburg_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mecklenburg_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
mecklenburg_merged = mecklenburg_data

# add clustering labels
mecklenburg_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mecklenburg_merged = mecklenburg_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

mecklenburg_merged

# Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mecklenburg_merged['Latitude'], mecklenburg_merged['Longitude'], mecklenburg_merged['Neighborhood'], mecklenburg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
      
from folium.plugins import FloatImage
image_file = 'http://www.carolana.com/NC/Counties/Images/Mecklenburg_County_NC_Map.jpg'

FloatImage(image_file, bottom=0, left=70).add_to(map_clusters)
map_clusters


import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
#mecklenburg_merged.hist(column='Density')
mecklenburg_merged.plot(kind='bar',x='Neighborhood',y='Density', color=rainbow[cluster])
plt.show()

# Dear reviewer please note that the cells are not run only the explaination and code is shown because it would make the notebook very long please see the reports for the result. Thank you have a nice day.